In [1]:
from manim import *

# 2. Dynamic Scenes

### Value Tracker and Updater

The keys to animating actual physics are ``ValueTracker`` and ``Updater``.

* ``ValueTracker`` provide an animation-wide real number that can be run from one value to another and subsequently be accessed globally in the scene. 
* ``Updater`` take a mobject and apply the defined operations for every animation frame. It is convenient to access the (global) value of a ``ValueTracker`` within the ``Updater`` and then for instance update the position of the mobject according to the desired behaviour under a tracker value.

In [43]:
%%manim -ql --fps 60 -v WARNING --disable_caching tracker_Scene


def x_func(t):
	if t < 4:
		return t
	elif t < 6:
		return -2 * (t - 4) + 4
	elif t < 10:
		return 0
	else:
		return (t - 10)


class tracker_Scene(Scene):
	def construct(self):
		headline = Title("Value Tracker and Updater").align_on_border(UP)
		self.add(headline)

		# defining the system size and range
		x_length = 10
		y_length = 4
		x_range = [0, 20, 2]
		y_range = [0, 10, 2]

		# defining the coordinate system as a number plane
		npla = NumberPlane(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, 
			background_line_style = {"stroke_opacity": 0.5}, 
			axis_config = {"include_numbers": True,}
			).move_to([0, -0.5, 0])

		npla_xlabel = npla.get_x_axis_label(Tex(r"t", font_size = 36))
		npla_ylabel = npla.get_y_axis_label(Tex(r"x", font_size = 36))
		npla_group = VGroup(npla, npla_xlabel, npla_ylabel)
		self.add(npla_group)

		# placing a dot in the coordinate systems origin
		dot_position = npla.c2p(0, 0)
		dot = Dot(radius = 0.1, color = RED).move_to(dot_position)
		self.add(dot)


		# defining the updater: moves the dot the x(t) position for every animation frame
		def dot_updater(dot):
			t = t_tracker.get_value()
			x = x_func(t)
			dot_position = npla.c2p(t, x)
			dot.move_to(dot_position)


		# defining a value tracker with initial value 0
		t_tracker = ValueTracker(0)
		# adding the updater to the dot
		dot.add_updater(dot_updater)
		# playing the value tracker over 10 seconds: specifying the rate function to run linearly
		self.wait(0.5)
		self.play(t_tracker.animate.set_value(20), rate_func = linear, run_time = 10)
		self.wait(3)

Manim Community v0.19.0